In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                resp = requests.get('https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum='+input_.loc[i, 'Part_Number']+'&brand='+input_.loc[i, 'Brand']+'&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960',
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                if len(resp) == 0:
                    src = ''
                else:
                    src = resp[0]['image_BG_Url']
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'Part_Number': input_.loc[i, 'Part_Number'],
                                         'Brand': input_.loc[i, 'Brand'],
                                         'Src': src}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[i, 'Part_Number'],
                                     'Brand': input_.loc[i, 'Brand']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Part_Number'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Part_Number'], ascending=[True])
output_correct.to_excel('./src-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number'], ascending=[True])
    output_error.to_excel('./src_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：548

73102  <->  [ok] - 剩余数量：528 - 当前时间：18:23:40
73099  <->  [ok] - 剩余数量：527 - 当前时间：18:23:40
73091  <->  [ok] - 剩余数量：526 - 当前时间：18:23:40
73088  <->  [ok] - 剩余数量：525 - 当前时间：18:23:40
73092  <->  [ok] - 剩余数量：524 - 当前时间：18:23:40
73090  <->  [ok] - 剩余数量：523 - 当前时间：18:23:40
37530  <->  [ok] - 剩余数量：522 - 当前时间：18:23:41
73104  <->  [ok] - 剩余数量：521 - 当前时间：18:23:41
73095  <->  [ok] - 剩余数量：520 - 当前时间：18:23:41
73105  <->  [ok] - 剩余数量：519 - 当前时间：18:23:41
73096  <->  [ok] - 剩余数量：518 - 当前时间：18:23:41
73107  <->  [ok] - 剩余数量：517 - 当前时间：18:23:41
73103  <->  [ok] - 剩余数量：516 - 当前时间：18:23:41
73108  <->  [ok] - 剩余数量：515 - 当前时间：18:23:41
73100  <->  [ok] - 剩余数量：514 - 当前时间：18:23:42
73089  <->  [ok] - 剩余数量：513 - 当前时间：18:23:42
73000  <->  [ok] - 剩余数量：512 - 当前时间：18:23:42
37540  <->  [ok] - 剩余数量：511 - 当前时间：18:23:42
73110  <->  [ok] - 剩余数量：510 - 当前时间：18:23:42
73111  <->  [ok] - 剩余数量：509 - 当前时间：18:23:42
73116  <->  [ok] - 剩余数量：508 - 当前时间：18:23:42
73001  <->  [ok] - 剩余数量：507 - 当前时间：18:23:42
73097  <->  [ok] - 剩余数量

73014  <->  [ok] - 剩余数量：340 - 当前时间：18:24:11
73319  <->  [ok] - 剩余数量：339 - 当前时间：18:24:12
73318  <->  [ok] - 剩余数量：338 - 当前时间：18:24:12
73321  <->  [ok] - 剩余数量：337 - 当前时间：18:24:12
73320  <->  [ok] - 剩余数量：336 - 当前时间：18:24:12
73300  <->  [ok] - 剩余数量：335 - 当前时间：18:24:12
73017  <->  [ok] - 剩余数量：334 - 当前时间：18:24:13
73322  <->  [ok] - 剩余数量：333 - 当前时间：18:24:13
73009  <->  [ok] - 剩余数量：332 - 当前时间：18:24:13
73324  <->  [ok] - 剩余数量：331 - 当前时间：18:24:13
73016  <->  [ok] - 剩余数量：330 - 当前时间：18:24:13
73325  <->  [ok] - 剩余数量：329 - 当前时间：18:24:13
73323  <->  [ok] - 剩余数量：328 - 当前时间：18:24:14
73329  <->  [ok] - 剩余数量：327 - 当前时间：18:24:14
73328  <->  [ok] - 剩余数量：326 - 当前时间：18:24:14
73327  <->  [ok] - 剩余数量：325 - 当前时间：18:24:14
73331  <->  [ok] - 剩余数量：324 - 当前时间：18:24:14
73330  <->  [ok] - 剩余数量：323 - 当前时间：18:24:14
37532  <->  [ok] - 剩余数量：322 - 当前时间：18:24:15
73335  <->  [ok] - 剩余数量：321 - 当前时间：18:24:15
73333  <->  [ok] - 剩余数量：320 - 当前时间：18:24:15
73296  <->  [ok] - 剩余数量：319 - 当前时间：18:24:15
73338  <->  [ok] - 剩余数量：318 - 当前

73521  <->  [ok] - 剩余数量：152 - 当前时间：18:24:49
73519  <->  [ok] - 剩余数量：151 - 当前时间：18:24:49
73515  <->  [ok] - 剩余数量：150 - 当前时间：18:24:50
73522  <->  [ok] - 剩余数量：149 - 当前时间：18:24:50
73536  <->  [ok] - 剩余数量：148 - 当前时间：18:24:50
73496  <->  [ok] - 剩余数量：147 - 当前时间：18:24:50
73523  <->  [ok] - 剩余数量：146 - 当前时间：18:24:50
73539  <->  [ok] - 剩余数量：145 - 当前时间：18:24:50
73033  <->  [ok] - 剩余数量：144 - 当前时间：18:24:51
73546  <->  [ok] - 剩余数量：143 - 当前时间：18:24:51
73541  <->  [ok] - 剩余数量：142 - 当前时间：18:24:52
73545  <->  [ok] - 剩余数量：141 - 当前时间：18:24:52
73547  <->  [ok] - 剩余数量：140 - 当前时间：18:24:52
73540  <->  [ok] - 剩余数量：139 - 当前时间：18:24:53
73542  <->  [ok] - 剩余数量：138 - 当前时间：18:24:53
73549  <->  [ok] - 剩余数量：137 - 当前时间：18:24:53
73551  <->  [ok] - 剩余数量：136 - 当前时间：18:24:54
73439  <->  [ok] - 剩余数量：135 - 当前时间：18:24:54
73548  <->  [ok] - 剩余数量：134 - 当前时间：18:24:54
73553  <->  [ok] - 剩余数量：133 - 当前时间：18:24:55
73022  <->  [ok] - 剩余数量：132 - 当前时间：18:24:55
73035  <->  [ok] - 剩余数量：131 - 当前时间：18:24:55
73524  <->  [ok] - 剩余数量：130 - 当前